In [1]:

import numpy as np
import miniball
import matplotlib.pyplot as plt
import pandas as pd
from tqdm.notebook import tqdm

from ML2_lib import loss
from ML2_lib import noise
from ML2_lib import algo_sgd
from  ML2_lib import merge
from ML2_lib import DC_SGD

In [2]:
X_mean = 5
X_var = 2
w_star = np.array([1])
w_star = w_star.reshape(1,-1)
d = w_star.shape[1]
n = 1000
c = 5
rng = np.random.default_rng()
w_init = w_star + rng.uniform(-c,c,size=d)

son = loss.LinearQuadraticLoss()
fuga = DC_SGD.DCSGD(w_star=w_star,n=n,E_var=2.2,X_mean=X_mean,X_var=X_var,noise="normal",loss_type=son,c=c)

In [3]:
fuga.learn(k=10,w_init=w_init)

(array([[0.89289677]]),
 0.3326619235223376,
 [[array([[4.51414873]]),
   array([[2.6481577]]),
   array([[2.13406183]]),
   array([[2.0455812]]),
   array([[1.65340204]]),
   array([[0.96777509]]),
   array([[0.98626003]]),
   array([[1.01646048]]),
   array([[0.87525783]]),
   array([[0.77393934]]),
   array([[0.75659249]]),
   array([[0.98382469]]),
   array([[0.93221261]]),
   array([[0.86250179]]),
   array([[0.72607951]]),
   array([[0.94834673]]),
   array([[0.91065834]]),
   array([[0.93758727]]),
   array([[0.85398637]]),
   array([[0.91390513]]),
   array([[0.96922002]]),
   array([[0.93004956]]),
   array([[0.80340375]]),
   array([[0.8644076]]),
   array([[0.7497304]]),
   array([[0.77709665]]),
   array([[0.81436322]]),
   array([[0.76782296]]),
   array([[0.84412768]]),
   array([[0.92606897]]),
   array([[0.79980082]]),
   array([[0.94624109]]),
   array([[0.86405332]]),
   array([[0.86299465]]),
   array([[0.7986598]]),
   array([[0.91832885]]),
   array([[0.91071907]])

In [4]:
fuga.trial_k(max_k=10)


(array([[[1.14521257]],
 
        [[1.17012672]],
 
        [[1.04076602]],
 
        [[1.10900684]],
 
        [[0.96625352]],
 
        [[0.80320388]],
 
        [[1.04297665]],
 
        [[0.97693298]],
 
        [[0.9309217 ]],
 
        [[0.96968071]]]),
 array([0.61151403, 0.83934991, 0.04819418, 0.34459223, 0.03302593,
        1.12313264, 0.05356279, 0.01543054, 0.13838254, 0.02665853]))

In [5]:
fuga.many_trails(trial_num=10,max_k=10)

  0%|          | 0/10 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [7]:
_,_,j =fuga.learn(k=10,w_init=w_init)

In [14]:
j

[[array([[4.51414873]]),
  array([[3.48154049]]),
  array([[2.1429708]]),
  array([[2.09831223]]),
  array([[1.70417877]]),
  array([[1.63168668]]),
  array([[1.37154815]]),
  array([[1.28375172]]),
  array([[1.121084]]),
  array([[0.74537745]]),
  array([[0.66421733]]),
  array([[0.51436508]]),
  array([[0.55485416]]),
  array([[0.6544925]]),
  array([[0.6987768]]),
  array([[0.72443011]]),
  array([[0.74424171]]),
  array([[0.81467439]]),
  array([[0.80859152]]),
  array([[1.06842212]]),
  array([[1.06782575]]),
  array([[1.05264306]]),
  array([[1.25209342]]),
  array([[1.16740761]]),
  array([[1.20985222]]),
  array([[1.20373393]]),
  array([[1.1911457]]),
  array([[0.9987451]]),
  array([[0.86973345]]),
  array([[0.82735102]]),
  array([[0.83039093]]),
  array([[1.04155786]]),
  array([[0.92048458]]),
  array([[1.15295678]]),
  array([[1.23160352]]),
  array([[1.27566128]]),
  array([[1.21415628]]),
  array([[1.29487852]]),
  array([[1.13916098]]),
  array([[1.22170117]]),
  array

In [16]:
j = np.array(j)